In [1]:
!pip install -qU langchain
!pip install -qU langchain-google-genai
!pip install -qU langchain-huggingface
!pip install -qU langchain-qdrant
!pip install -qU langchain-community
!pip install -qU langgraph
!pip install fastembed
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.

In [38]:
import requests
from bs4 import BeautifulSoup
import os
from google.colab import userdata
import xml.etree.ElementTree as ET
from tqdm import tqdm
from langchain.schema import Document

In [39]:
os.environ["GOOGLE_API_KEY"] = userdata.get('gemini_api_key')
os.environ["LANGSMITH_API_KEY"] = userdata.get('langsmith_api_key')
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["QDRANT_API_KEY"] = userdata.get('qdrant_api_key')

In [40]:
def get_all_sitemap_links(index_url: str) -> list[str]:
    res = requests.get(index_url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "xml")

    links = soup.find_all("loc")
    sitemap_urls = [
        loc.text.strip()
        for loc in links
        if not loc.text.strip().endswith("sitemap-root.xml")
    ]
    return sitemap_urls

index_url = "https://www.cp.eng.chula.ac.th/sitemap.xml"
sitemap_urls = get_all_sitemap_links(index_url)

In [41]:
sitemap_urls

['https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2025.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2024.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2023.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2022.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2021.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2020.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2019.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2018.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2017.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2016.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2015.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-post.2014.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-posttype-page.xml',
 'https://www.cp.eng.chula.ac.th/sitemap-author.xml']

In [42]:
def parse_sitemap(sitemap_url):
    response = requests.get(sitemap_url)
    response.raise_for_status()  # ensure the request was successful
    root = ET.fromstring(response.content)
    urls = []
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    for url in root.findall('ns:url', namespace):
        loc = url.find('ns:loc', namespace).text
        urls.append(loc)
    return urls

urls = []
for sitemap_url in tqdm(sitemap_urls):
    urls += parse_sitemap(sitemap_url)

  0%|          | 0/14 [00:00<?, ?it/s]


ConnectionError: HTTPSConnectionPool(host='www.cp.eng.chula.ac.th', port=443): Max retries exceeded with url: /sitemap-posttype-post.2025.xml (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7d6149c9ebd0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [24]:
def scrape_cp_generic(url: str) -> Document:
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    content_div = soup.select_one("div#content")
    if not content_div:
        raise ValueError(f"No #content found in {url}")

    # Extract text and clean up
    body = content_div.get_text(separator="\n", strip=True)
    h1_post = soup.select_one("h1.title-post")
    if h1_post:
        title = h1_post.text.strip()
    elif soup.title:
        title = soup.title.string.strip()
    else:
        title = "Untitled"

    metadata = {
        "source": url,
        "title": title,
    }

    return Document(page_content=body, metadata=metadata)


In [ ]:
docs = []

for url in tqdm(urls, desc="Scraping URLs", position=0, leave=True):
    try:
        doc = scrape_cp_generic(url)
        docs.append(doc)
    except ValueError as e:
        print(f"[SKIP] {e}")
    except Exception as e:
        print(f"[ERROR] Unexpected error on {url}: {e}")

Scraping URLs: 100%|██████████| 9/9 [00:12<00:00,  1.34s/it]


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE",model_kwargs=model_kwargs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="Alibaba-NLP/gte-reranker-modernbert-base")

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/598M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # number of characters
    chunk_overlap=100,    # to retain context between chunks
    separators=["\n\n", "\n", ".", " "],
)

split_docs = text_splitter.split_documents(docs)

In [ ]:
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

arabic.txt:   0%|          | 0.00/6.35k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

catalan.txt:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

bengali.txt:   0%|          | 0.00/5.44k [00:00<?, ?B/s]

...

slovene.txt:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

tajik.txt:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

spanish.txt:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
# Set your Qdrant endpoint and API key
client = QdrantClient(
    url="https://ff9ebd5e-31f8-4e7a-a866-ef23b9b9cd5e.us-west-2-0.aws.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"],
)

client.recreate_collection(
    collection_name="cp-genie",
    vectors_config={"dense": VectorParams(size=768, distance=Distance.COSINE)},
    sparse_vectors_config={
        "sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))
    },
)


<ipython-input-13-9bfba98708f2>:7: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
qdrant = QdrantVectorStore(
    client=client,
    collection_name="cp-genie",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse",
)

qdrant.add_documents(documents=split_docs)

['a8fd1026a6ff4ca49320f72d04f3fbe6',
 '7431a8e93b9d441cb3ff9f336c328da2',
 'fc22037edb1c4db9ae1ba33bb8b09853',
 '0d16f1f96d264778ac0b381282445902',
 '338c7c6cf5a2431dbe828f1c7b0c4e1c',
 '171de35492864568be16b1d3fad71ca4',
 'afa8981dcc174132be81d8d2bc51b33d',
 '806eff1764e74532bd002cdcb01a9ebb',
 'd2cd4a2a8a924eb0a99ca143de2315b9',
 'fd23563805694147a65b2565bb68e972',
 '0cdb432d23ba433eb344b55b47e59905',
 '5e1ce5644dbc44dbb683ada486d585de',
 'a5834ed170d64764af0a8daf9c8d936c',
 'ff552b425f554668997bf0047f4ac688',
 'b34ad522c7d94151a9d773febe0348c0',
 '911783c6ba15418aa2da149604eec5d1',
 '61fa3eed525d401db1745c661fe6eb17',
 '0612953173ef420f9bea07e833429ad3',
 '36e5c3342a6749b891fe67d6d27230c0',
 '08ed48228956477180e4fcbca0725bc4',
 '126085e7829f42a09a33606e9f102d8b',
 '755d88938e0a48b29b2c11ab565d2a1c',
 '3b5a5a0ddda849bbb8095ca59a2a84f7',
 '1ea7b6d2017742d5b0c9304387c4e18e',
 'dc77a0ad4c3c4d6d9e552cd76acf34ea',
 'a05ad8d0ba554323884e6f9c6de7139e',
 '37590555d0ce402a8f8f97f7e78517a2',
 

In [ ]:
retriever = qdrant.as_retriever(search_type="similarity", k=10)

query = "ผศ.ดร.เกริก ภิรมย์โสภา ได้รางวัลอะไรบ้าง ช่วงไหน"
results = retriever.get_relevant_documents(query)

for doc in results:
    print(doc.metadata["title"])
    print(doc.page_content[:300])
    print("-" * 50)


ภาควิชาวิศวกรรมคอมพิวเตอร์ขอแสดงความยินดี กับ ผศ.ดร.เกริก ภิรมย์โสภา  รับรางวัลยกย่องเชิดชูเกียรติบุคลากรแห่งจุฬาลงกรณ์มหาวิทยาลัย
ภาควิชาวิศวกรรมคอมพิวเตอร์ขอแสดงความยินดี กับ ผศ.ดร.เกริก ภิรมย์โสภา 
รับรางวัลยกย่องเชิดชูเกียรติบุคลากรแห่งจุฬาลงกรณ์มหาวิทยาลัย
วัน อังคาร ที่ 4 กันยายน 2561
ผศ.ดร.เกริก ภิรมย์โสภา รับรางวัลยกย่องเชิดชูเกียรติบุคลากรแห่งจุฬาลงกรณ์มหาวิทยาลัย
รางวัลระดับนานาชาติและระดับชาติประจำปี 2560 – 2561
--------------------------------------------------
ภาควิชาวิศวกรรมคอมพิวเตอร์ ขอแสดงความยินดี ผศ.ดร.เกริก  ภิรมย์โสภา
ภาควิชาวิศวกรรมคอมพิวเตอร์ ขอแสดงความยินดี ผศ.ดร.เกริก ภิรมย์โสภาเป็นผู้ได้รับรางวัลยกย่องเชิดชูเกียรติอาจารย์ด้านการเรียนการสอน กองทุนกาญจนาภิเษกเฉลิมพระเกียรติ ประจำปี 2558
ได้รับรางวัลระดับดีเด่น สาขาวิทยาศาสตร์และเทคโนโลยี
--------------------------------------------------
ภาควิชาวิศวกรรมคอมพิวเตอร์ ขอแสดงความยินดี ผศ.ดร.เกริก ภิรมย์โสภาและทีม กับรางวัล BRONZE PRIZE
ภาควิชาวิศวกรรมคอมพิวเตอร์ ขอแสดงความยินดี
 ผศ.ดร.เกริก ภิรมย์โสภาและทีม กับรางวัล

<ipython-input-17-2ceb97909024>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [ ]:
context = ""
for idx, doc in enumerate(results):
    title = doc.metadata["title"]
    content = doc.page_content
    sums_str = f"""
Title: {title}
Content: {content}
      """
    context += sums_str + '\n'

input_text = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"

prompt = """Perform an question answering with the corresponding Context, Question and Answer. 
***You do not need to specify which Document you get from*** 
"""

In [ ]:
print(input_text)

Context:
Document 1
Title: ภาควิชาวิศวกรรมคอมพิวเตอร์ขอแสดงความยินดี กับ ผศ.ดร.เกริก ภิรมย์โสภา  รับรางวัลยกย่องเชิดชูเกียรติบุคลากรแห่งจุฬาลงกรณ์มหาวิทยาลัย
Date: September 4, 2018
Tags: ความสำเร็จ
Content: ภาควิชาวิศวกรรมคอมพิวเตอร์ขอแสดงความยินดี กับ ผศ.ดร.เกริก ภิรมย์โสภา 
รับรางวัลยกย่องเชิดชูเกียรติบุคลากรแห่งจุฬาลงกรณ์มหาวิทยาลัย
วัน อังคาร ที่ 4 กันยายน 2561
ผศ.ดร.เกริก ภิรมย์โสภา รับรางวัลยกย่องเชิดชูเกียรติบุคลากรแห่งจุฬาลงกรณ์มหาวิทยาลัย
รางวัลระดับนานาชาติและระดับชาติประจำปี 2560 – 2561
      
Document 2
Title: ภาควิชาวิศวกรรมคอมพิวเตอร์ ขอแสดงความยินดี ผศ.ดร.เกริก  ภิรมย์โสภา
Date: March 28, 2016
Tags: ความสำเร็จ
Content: ภาควิชาวิศวกรรมคอมพิวเตอร์ ขอแสดงความยินดี ผศ.ดร.เกริก ภิรมย์โสภาเป็นผู้ได้รับรางวัลยกย่องเชิดชูเกียรติอาจารย์ด้านการเรียนการสอน กองทุนกาญจนาภิเษกเฉลิมพระเกียรติ ประจำปี 2558
ได้รับรางวัลระดับดีเด่น สาขาวิทยาศาสตร์และเทคโนโลยี
      
Document 3
Title: ภาควิชาวิศวกรรมคอมพิวเตอร์ ขอแสดงความยินดี ผศ.ดร.เกริก ภิรมย์โสภาและทีม กับรางวัล BRONZE PRIZE
Date: Dece

In [ ]:
response = llm.predict(input_text)
print(response)

<ipython-input-20-3b5a5aea15c3>:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(input_text)


จากข้อมูลในเอกสาร ผศ.ดร.เกริก ภิรมย์โสภา ได้รับรางวัลดังนี้:

*   **2558:** รางวัลยกย่องเชิดชูเกียรติอาจารย์ด้านการเรียนการสอน กองทุนกาญจนาภิเษกเฉลิมพระเกียรติ ระดับดีเด่น สาขาวิทยาศาสตร์และเทคโนโลยี (Document 2)
*   **2017:** รางวัล BRONZE PRIZE จากงาน Seoul International Innovation Fair 2017 จากผลงาน "หุ่นยนต์แบบมีปฏิสัมพันธ์เพื่อการเรียนรู้เชิงรุก" (Document 3)
*   **2560 – 2561:** รางวัลยกย่องเชิดชูเกียรติบุคลากรแห่งจุฬาลงกรณ์มหาวิทยาลัย รางวัลระดับนานาชาติและระดับชาติ (Document 1)
*   **2018:** เป็นอาจารย์ที่ปรึกษาให้นายอภิรุจ ชุ่มวัฒนะ ซึ่งได้รับรางวัลหัวข้อพิเศษดีเด่น จากผลงานชื่อ เส้นทางสู่การหลบหนีของคนร้าย the direction of prison escapes (Document 4)
